# Simple RAM Simulator

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt    
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from ipywidgets import Button, HBox, VBox, Layout
from IPython.display import HTML

RAM is an acronym for Reliability Availability and Maintainability. The term is often used in the context of product design (concept selection), life cycle costing, maintenance, and safety assessments. There exist numerous methods for RAM analysis which can be broadly divided into analytical and simulation-based techniques. Limitations of analytical methods include constant failure rate assumption, repair time neglect, redundancy etc., and can be overcome by simulation methods. 

This notebook is a simulation-based RAM analysis tool which models a component (or system) as a repairable system. Note that the word component or system is interchangeably used depending on context. The simplest example of a system is a redundant configuration of components. 

The simulation method used is Discrete Event Simulation (DES). In DES, a component jumps from upstate to downstate in a simulation, based on discrete events i.e. Uptime, CM and PM. The time spent on each event is a random entity, defined by the user in the form of a probability distribution (see below for input parameters). Any run of a simulation is a sequence of events in time, and to aggregate the results multiple simulations are run. Thereofore, the name *Monte Carlo Next Event* simulation. 

Three type of components modelled in this workbook are: 

1. Simple series component
2. Redundant system with delayed repair policy
3. Redundant system with independent failure and repair 

The user is expected acquaintance with reliability assessment methods. 

The terminology used is in accordance with IEC Definitions, and can be accesed [here](http://www.electropedia.org/iev/iev.nsf/index?openform&part=192). All time based input parameters should be in the units of hours. The followng input parmeteres are required to run simulation:
1. **CMleft, CMmode and CMright:** *part of the maintenance time taken to perform corrective maintenance, including technical delays and logistic delays inherent in corrective maintenance* [IEC](http://www.electropedia.org/iev/iev.nsf/display?openform&ievref=192-07-07). The triangular distribution is used for modelling. 
2. **PMleft, PMmode and PMright:** *part of the maintenance time taken to perform preventive maintenance, including technical delays and logistic delays inherent in preventive maintenance* [IEC](http://www.electropedia.org/iev/iev.nsf/display?openform&ievref=192-07-05). The triangular distribution is used for modelling. 
3. **PMinterval:** Preventive maintenace frequency. For example 8760 means a preventive maintenance action every year.
4. **Wibull shape and scale:** Parameters of Weibull Distribution used to model time to failure event.
5. **PMtolerance:** A pramater to skip a scheduled PM action if CM was performed specified number of hours ago. 
6. **NumSimulations:** Replications of simulations
7. **SimPeriod:** Simulation period for the program.

The results are explained with each component type. 

In [16]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## 1. Simple Series Component

A simple series component brings the system down upon failure. 

The results shown are the aggregated results of given number of simulations in units of days. Following results are of interest:

1. **UptimeCount:** Count of times component spent as uptime in a simulation.  
2. **CMcount:** Count of times a component spent as CM in a simulation. 
3. **PMcount:** Count of times a component spent as PM in a simulation.
4. **UptimeSum:** Total time component spent as uptime in a simulation.
5. **CMSum:** Total time component spent as CM in a simulation.
6. **PMSUM:** Total time component spent as CM in a simulation.
7. **Downtime:** The sum of CM and PM time in a simulation.
8. **Replacements:** Total number of replacements due to PM or CM events.  
9. **MTBF:** UptimeSum / CMcount in a simulation. 
10. **MTTR:** Downtime / (CMcount + PMcount) in a simulation.
11. **Availability:** Steady state availability in a simulation given by MTBF / (MTBF + MTTR).

The simulation is run by entering the parameters below and hitting the *Run Simulation* button.

In [17]:
from funk_CallingSingleComp20072020 import funk_CallingSingleComp20072020 as SeriesCompSim

style = {'description_width': '150px'}

CMleft1 = widgets.BoundedFloatText(value=520, min=1, max=8760*3, step=24, description='CM left time (hrs):', disabled=False, style =style)
CMmode1 = widgets.BoundedFloatText(value=720, min=2, max=8760*3, step=24, description='CM mode time (hrs):', disabled=False, style =style)
CMright1 = widgets.BoundedFloatText(value=920, min=3, max=8760*3, step=24, description='CM right time (hrs):', disabled=False, style =style)

PMleft1 = widgets.BoundedFloatText(value=96, min=1, max=8760*3, step=24, description='PM left time (hrs):', disabled=False, style =style)
PMmode1 = widgets.BoundedFloatText(value=120, min=2, max=8760*3, step=24, description='PM mode time (hrs):', disabled=False, style =style)
PMright1 = widgets.BoundedFloatText(value=168, min=3, max=8760*3, step=24, description='PM right time (hrs):', disabled=False, style =style)

WeibullShape1 = widgets.BoundedFloatText(value=1.5, min=1, max=30, step=1, description='Shape parameter:', disabled=False, style =style)
WeibullScale1 = widgets.BoundedFloatText(value=8760*1.5, min=8760/2, max=876000, step=1, description='Scale parameter  (hrs):', disabled=False, style =style)
SimPeriod1 = widgets.BoundedIntText(value=87600, min=8760, max=876000, step=8760, description='Sim. Period (hrs):', disabled=False, style =style)
PMinterval1 = widgets.BoundedFloatText(value=8760*2, min=8760/4, max=876000, step=8760/2, description='PM freq  (hrs):', disabled=False, style =style)
PMtolerance1 = widgets.BoundedFloatText(value=720, min=1, max=8760*2, step=24, description='PM toler  (hrs):', disabled=False, style =style)
NumSimulations1 = widgets.BoundedIntText(value=100, min=100, max=1000, step=50, description='No of Simulation:', disabled=False, style =style)

Btn1 = widgets.Button(description='Run Simulation', disabled=False, button_style='success', tooltip='Click me', icon='', )
left_box = VBox([CMleft1, PMleft1,  PMinterval1, PMtolerance1, Btn1])
center_box = VBox([CMmode1, PMmode1, WeibullShape1, NumSimulations1])
right_box = VBox([CMright1, PMright1, WeibullScale1, SimPeriod1])

box_layout = Layout(display='flex', flex_flow='row', justify_content = 'center', border='solid 2px green', width='100%')
hbox1 = HBox([left_box,center_box, right_box], layout = box_layout)
output1 = widgets.Output()
display(hbox1, output1 )

def on_button_clicked(b):
    with output1:
        clear_output()
        SeriesCompSim(CMleft = CMleft1.value, CMmode = CMmode1.value, CMright = CMright1.value,
               PMleft = PMleft1.value, PMmode = PMmode1.value, PMright = PMright1.value,
               WeibullShape = WeibullShape1.value, WeibullScale = WeibullScale1.value, 
               SimPeriod = SimPeriod1.value, PMinterval = PMinterval1.value, PMtolerance = PMtolerance1.value, 
               NumSimulations = NumSimulations1.value)
        show_inline_matplotlib_plots()
        
Btn1.on_click(on_button_clicked)
#%matplotlib widget

Output()

## 2. Redundant Component with Delayed Repair Policy

This is a case of 1oo2 system i.e. system works as long as one out of two components are available (no dwontime until both fail). The meaning of delayed repair policy is that upon the failure of any component, repair is not initiated until both  components fail or a PM action restores the redundancy. 

The results shown are the aggregated results of given number of simulations in units of days. Following results are of interest:
1. **UptimeCount:** Count of times system spent as uptime in a simulation.  
2. **CMcount:** Count of times a system spent as CM in a simulation. 
3. **PMcount:** Count of times a system spent as PM in a simulation.
4. **UptimeSum:** Sum of times system spent as uptime in a simulation.
5. **CMSum:** Sum of CM times component spent as CM in a simulation.
6. **PMSUM:** Sum of PM times component spent as CM in a simulation.
7. **Downtime:** Sum of CM and PM time in a simulation.
8. **Replacements:** Total number of replacements due to PM or CM events. Since these maintenance events bring the system to as-good-as-new-state (i.e. restoration of redundancy), replacements in a simulation are calcualted as sum of 2*(CMcount + PMcount). 
9. **MTBF:** (UptimeSum / CMcount) in a simulation. 
10. **MTTR:** Downtime / (CMcount + PMcount) in a simulation.
11. **Availability:** Steady state availability in a simulation given by MTBF / (MTBF + MTTR).

The simulation is run by entering the parameters below and hitting the Run Simulation button.

In [18]:
from funk_CallingRedDelayRepair20072020 import funk_CallingRedDelayRepair20072020 as RedDelayRepair

style = {'description_width': '150px'}

CMleft2 = widgets.BoundedFloatText(value=520, min=1, max=8760*3, step=24, description='CM left time (hrs):', disabled=False, style =style)
CMmode2 = widgets.BoundedFloatText(value=720, min=2, max=8760*3, step=24, description='CM mode time (hrs):', disabled=False, style =style)
CMright2 = widgets.BoundedFloatText(value=920, min=3, max=8760*3, step=24, description='CM right time (hrs):', disabled=False, style =style)

PMleft2 = widgets.BoundedFloatText(value=96, min=1, max=8760*3, step=24, description='PM left time (hrs):', disabled=False, style =style)
PMmode2 = widgets.BoundedFloatText(value=120, min=2, max=8760*3, step=24, description='PM mode time (hrs):', disabled=False, style =style)
PMright2 = widgets.BoundedFloatText(value=168, min=3, max=8760*3, step=24, description='PM right time (hrs):', disabled=False, style =style)

WeibullShape2 = widgets.BoundedFloatText(value=1.5, min=1, max=30, step=1, description='Shape parameter:', disabled=False, style =style)
WeibullScale2 = widgets.BoundedFloatText(value=8760*1.5, min=8760/2, max=876000, step=1, description='Scale parameter  (hrs):', disabled=False, style =style)
SimPeriod2 = widgets.BoundedIntText(value=87600, min=8760, max=876000, step=8760, description='Sim. Period (hrs):', disabled=False, style =style)
PMinterval2 = widgets.BoundedFloatText(value=8760*2, min=8760/4, max=876000, step=8760/2, description='PM freq  (hrs):', disabled=False, style =style)
PMtolerance2 = widgets.BoundedFloatText(value=720, min=1, max=8760*2, step=24, description='PM toler  (hrs):', disabled=False, style =style)
NumSimulations2 = widgets.BoundedIntText(value=100, min=10, max=1000, step=50, description='No of Simulation:', disabled=False, style =style)

Btn2 = widgets.Button(description='Run Simulation', disabled=False, button_style='success', tooltip='Click me', icon='', )
left_box2 = VBox([CMleft2, PMleft2,  PMinterval1, PMtolerance1, Btn2])
center_box2 = VBox([CMmode2, PMmode2, WeibullShape2, NumSimulations2])
right_box2 = VBox([CMright2, PMright2, WeibullScale2, SimPeriod2])

box_layout = Layout(display='flex', flex_flow='row', justify_content = 'center', border='solid 2px green', width='100%')
hbox2 = HBox([left_box2, center_box2, right_box2], layout = box_layout)
output2 = widgets.Output()
display(hbox2, output2)

def on_button_clicked(b):
    with output2:
        clear_output()
        RedDelayRepair(CMleft = CMleft2.value, CMmode = CMmode2.value, CMright = CMright2.value,
               PMleft = PMleft2.value, PMmode = PMmode2.value, PMright = PMright2.value,
               WeibullShape = WeibullShape2.value, WeibullScale = WeibullScale2.value, 
               SimPeriod = SimPeriod2.value, PMinterval = PMinterval2.value, PMtolerance = PMtolerance2.value, 
               NumSimulations = NumSimulations2.value)
        show_inline_matplotlib_plots()
        
Btn2.on_click(on_button_clicked)
#%matplotlib widget

Output()

## 3. Redundant System with Independent Components

This is another case of 1oo2 system i.e. system works if one out of two components are available (no downtime until both fail). The independent failure and repair mean that the failure of one does not affect the operation or repair of other. That is the failure and repair probabilities of both components are independent of each other. Note that the independence assumption is not very restrictive and employed frequently in classical reliability modelling. 


The results shown are the aggregated results of given number of simulations in units of days. Following results are of interest:
1. **TotalPMcount:** Number of times PM is performed on component 1 or componnent 2 in a simulation.
2. **TotalCMcount:** Number of times CM is performed on component 1 or componnent 2 in a simulation.
3. **DowntimeCount:** Number of times CM or PM is performed simultaneously on component 1 and component in a simulation.
4. **DowntimeCount_CMonly:** Number of times CM is performed simultaneously on both components in a simulation.
5. **Replacements:** Number of repalcements due to CM or PM in a simulation. 
6. **Uptime:** Number of instances when either of the component is available in a simulation. 
7. **Downtime:** Number of instance when both components are simultaneosuly unavailable in a simulation due to CM or PM. 
8. **MTBF:** UptimeSum / DowntimeCount_CMonly in a simulation. 
9. **MTTR:** Downtime / DowntimeCount in a simulation
10. **Availability:** Steady state availability in a simulation given by MTBF / (MTBF + MTTR).

The simulation is run by entering the parameters below and hitting the Run Simulation button.

In [19]:
from funk_CallingRed_Indep20072020 import RedComp_IndepSim as RedunIndepSim

style = {'description_width': '150px'}
CompType3 = widgets.BoundedFloatText(value=1, min=1, max=2, step=1, description='Series or Redund:', disabled=False)

CMleft3 = widgets.BoundedFloatText(value=520, min=1, max=8760*3, step=24, description='CM left time (hrs):', disabled=False, style =style)
CMmode3 = widgets.BoundedFloatText(value=720, min=2, max=8760*3, step=24, description='CM mode time (hrs):', disabled=False, style =style)
CMright3 = widgets.BoundedFloatText(value=920, min=3, max=8760*3, step=24, description='CM right time (hrs):', disabled=False, style =style)

PMleft3 = widgets.BoundedFloatText(value=96, min=1, max=8760*3, step=24, description='PM left time (hrs):', disabled=False, style =style)
PMmode3 = widgets.BoundedFloatText(value=120, min=2, max=8760*3, step=24, description='PM mode time (hrs):', disabled=False, style =style)
PMright3 = widgets.BoundedFloatText(value=168, min=3, max=8760*3, step=24, description='PM right time (hrs):', disabled=False, style =style)

WeibullShape3 = widgets.BoundedFloatText(value=1.5, min=1, max=10, step=1, description='Shape parameter:', disabled=False, style =style)
WeibullScale3 = widgets.BoundedFloatText(value=8760*1.5, min=8760/2, max=876000, step=1, description='Scale parameter  (hrs):', disabled=False, style =style)
SimPeriod3 = widgets.BoundedIntText(value=87600, min=8760, max=876000, step=8760, description='Sim. Period (hrs):', disabled=False, style =style)
PMinterval3 = widgets.BoundedFloatText(value=8760*2, min=8760/4, max=876000, step=8760/2, description='PM freq  (hrs):', disabled=False, style =style)
PMtolerance3 = widgets.BoundedFloatText(value=720, min=1, max=8760*2, step=24, description='PM toler  (hrs):', disabled=False, style =style)
NumSimulations3 = widgets.BoundedIntText(value=100, min=100, max=1000, step=50, description='No of Simulation:', disabled=False, style =style)

Btn3 = widgets.Button(description='Run Simulation', disabled=False, button_style='success', tooltip='Click me', icon='', )
left_box3 = VBox([CMleft3, PMleft3,  PMinterval3, PMtolerance3, Btn3])
center_box3 = VBox([CMmode3, PMmode3, WeibullShape3, NumSimulations3])
right_box3 = VBox([CMright3, PMright3, WeibullScale3, SimPeriod3])

box_layout = Layout(display='flex', flex_flow='row', justify_content = 'center', border='solid 2px green', width='100%')
hbox3 = HBox([left_box3, center_box3, right_box3], layout = box_layout) 
output3 = widgets.Output()
display(hbox3, output3 )

def on_button_clicked(b):
    with output3:
        clear_output()
        RedunIndepSim(CMleft = CMleft3.value, CMmode = CMmode3.value, CMright = CMright3.value,
               PMleft = PMleft3.value, PMmode = PMmode3.value, PMright = PMright3.value,
               WeibullShape = WeibullShape3.value, WeibullScale = WeibullScale3.value, 
               SimPeriod = SimPeriod3.value, PMinterval = PMinterval3.value, PMtolerance = PMtolerance3.value, 
                      NumSimulations = NumSimulations3.value)
        show_inline_matplotlib_plots()
        
Btn3.on_click(on_button_clicked)
#%matplotlib widget

Output()

## 4. Distribution Viewer

Move the sliders below to visulaize the distributions.

### Triangular Distribution

In [20]:
@interact
def make_Hist(a=1, b=750, c=1500):
    Triangular = np.array(np.random.triangular(a, b, c, size = 10000));
    plt.hist(Triangular);

interactive(children=(IntSlider(value=1, description='a', max=3, min=-1), IntSlider(value=750, description='b'…

### Weibull Distribution

In [21]:
@interact
def make_Hist(shape=1.5, scale=21900):
    Weibull = np.array(scale * np.random.weibull(shape, size = 10000));
    plt.hist(Weibull);

interactive(children=(FloatSlider(value=1.5, description='shape', max=4.5, min=-1.5), IntSlider(value=21900, d…